In [1]:
import argparse
import copy
import json
import os
import sys
import numpy as np
import torch
import yaml
from det3d import torchie
# from det3d.datasets import build_dataloader, build_dataset
from det3d.models import build_detector
from det3d.torchie import Config
from det3d.torchie.apis import (
    batch_processor,
    build_optimizer,
    get_root_logger,
    init_dist,
    set_random_seed,
    train_detector,
)
from det3d.torchie.trainer import get_dist_info, load_checkpoint
from det3d.torchie.trainer.utils import all_gather, synchronize
from torch.nn.parallel import DistributedDataParallel
import pickle 
import time 

[Initialized] name of Registry  reader
[Initialized] len of _module_dict  0
[Initialized] name of Registry  backbone
[Initialized] len of _module_dict  0
[Initialized] name of Registry  neck
[Initialized] len of _module_dict  0
[Initialized] name of Registry  head
[Initialized] len of _module_dict  0
[Initialized] name of Registry  loss
[Initialized] len of _module_dict  0
[Initialized] name of Registry  detector
[Initialized] len of _module_dict  0
[Initialized] name of Registry  second_stage
[Initialized] len of _module_dict  0
[Initialized] name of Registry  roi_head
[Initialized] len of _module_dict  0
registration of classes:  SpMiddleResNetFHD
registration of classes:  CenterHead
registration of classes:  SingleStageDetector
registration of classes:  PointPillars
registration of classes:  VoxelNet
registration of classes:  TwoStageDetector
registration of classes:  RPN
registration of classes:  PillarFeatureNet
registration of classes:  PointPillarsScatter
registration of classes

In [2]:
def save_pred(pred, root):
    with open(os.path.join(root, "prediction.pkl"), "wb") as f:
        pickle.dump(pred, f)


In [3]:
CFG_NAME='waymo_centerpoint_voxelnet_3x'


def parse_args():
    parser = argparse.ArgumentParser(description="Train a detector")
    parser.add_argument("--config", 
                        default=f'./tools/dist_test.py configs/waymo/voxelnet/two_stage/{CFG_NAME}.py', help="train config file path")
    parser.add_argument("--work_dir", 
                        default=f'work_dirs/{CFG_NAME}',
                        # required=True, 
                        help="the dir to save logs and models")
    parser.add_argument(
        "--checkpoint", 
        default=f'work_dirs/{CFG_NAME}/latest.pth',
        help="the dir to checkpoint which the model read from"
    )
    parser.add_argument(
        "--txt_result",
        type=bool,
        default=False,
        help="whether to save results to standard KITTI format of txt type",
    )
    parser.add_argument(
        "--gpus",
        type=int,
        default=1,
        help="number of gpus to use " "(only applicable to non-distributed training)",
    )
    parser.add_argument(
        "--launcher",
        choices=["none", "pytorch", "slurm", "mpi"],
        default="none",
        help="job launcher",
    )
    parser.add_argument("--speed_test", action="store_true")
    parser.add_argument("--local_rank", type=int, default=0)
    parser.add_argument("--testset", action="store_true")

    args = parser.parse_args()
    if "LOCAL_RANK" not in os.environ:
        os.environ["LOCAL_RANK"] = str(args.local_rank)

    return args


In [4]:
import sys

# Simulate command-line arguments
sys.argv = [
    'script_name.py',  # Typically the script name, can be anything
    '--config', '../configs/waymo/voxelnet/two_stage/waymo_centerpoint_voxelnet_graphrcnn_6epoch_freeze.py',
    '--work_dir', '../work_dirs/waymo_centerpoint_voxelnet_graphrcnn_6epoch_freeze',
    '--checkpoint', '/home/rajeev-gupta/sensyn_ws/src/GraphRCNN/work_dirs/waymo_centerpoint_voxelnet_graphrcnn_6epoch_freeze/latest.pth',
    # '--txt_result', 'True',
    '--gpus', '1',
    '--local_rank', '0',
    # '--speed_test'
]

# Call the parse_args function
args = parse_args()

# Check the parsed arguments
print(args)

cfg = Config.fromfile(args.config)
cfg.local_rank = args.local_rank


Namespace(checkpoint='/home/rajeev-gupta/sensyn_ws/src/GraphRCNN/work_dirs/waymo_centerpoint_voxelnet_graphrcnn_6epoch_freeze/latest.pth', config='../configs/waymo/voxelnet/two_stage/waymo_centerpoint_voxelnet_graphrcnn_6epoch_freeze.py', gpus=1, launcher='none', local_rank=0, speed_test=False, testset=False, txt_result=False, work_dir='../work_dirs/waymo_centerpoint_voxelnet_graphrcnn_6epoch_freeze')


In [5]:

distributed = False if args.launcher == 'none' else True

if distributed:
    torch.cuda.set_device(args.local_rank)
    torch.distributed.init_process_group(backend="nccl", init_method="env://")

    cfg.gpus = torch.distributed.get_world_size()
else:
    cfg.gpus = args.gpus


In [6]:

# init logger before other steps
logger = get_root_logger(cfg.log_level)
logger.info("Distributed testing: {}".format(distributed))
logger.info(f"torch.backends.cudnn.benchmark: {torch.backends.cudnn.benchmark}")


2024-06-13 17:00:28,594 - INFO - Distributed testing: False
2024-06-13 17:00:28,595 - INFO - torch.backends.cudnn.benchmark: False


In [7]:

model = build_detector(cfg.model, train_cfg=None, test_cfg=cfg.test_cfg)



Executed build_detector
[In build_detector]name of Registry  detector
[In build_detector]_module_dict  {'SingleStageDetector': <class 'det3d.models.detectors.single_stage.SingleStageDetector'>, 'PointPillars': <class 'det3d.models.detectors.point_pillars.PointPillars'>, 'VoxelNet': <class 'det3d.models.detectors.voxelnet.VoxelNet'>, 'TwoStageDetector': <class 'det3d.models.detectors.two_stage.TwoStageDetector'>}

***************************model is build_from_cfg***************************
obj_type is str:  TwoStageDetector
[In build_from_cfg] building  <class 'det3d.models.detectors.two_stage.TwoStageDetector'>


Executed build_detector
[In build_detector]name of Registry  detector
[In build_detector]_module_dict  {'SingleStageDetector': <class 'det3d.models.detectors.single_stage.SingleStageDetector'>, 'PointPillars': <class 'det3d.models.detectors.point_pillars.PointPillars'>, 'VoxelNet': <class 'det3d.models.detectors.voxelnet.VoxelNet'>, 'TwoStageDetector': <class 'det3d.models.

2024-06-13 17:00:28,655 - INFO - Finish RPN Initialization
2024-06-13 17:00:28,656 - INFO - num_classes: [3]
2024-06-13 17:00:28,660 - INFO - Finish CenterHead Initialization
2024-06-13 17:00:28,702 - INFO - bbox_head.shared_conv.0.bias                   loaded from bbox_head.shared_conv.0.bias                               of shape (64,)
2024-06-13 17:00:28,702 - INFO - bbox_head.shared_conv.0.weight                 loaded from bbox_head.shared_conv.0.weight                             of shape (64, 512, 3, 3)
2024-06-13 17:00:28,702 - INFO - bbox_head.shared_conv.1.bias                   loaded from bbox_head.shared_conv.1.bias                               of shape (64,)
2024-06-13 17:00:28,703 - INFO - bbox_head.shared_conv.1.running_mean           loaded from bbox_head.shared_conv.1.running_mean                       of shape (64,)
2024-06-13 17:00:28,703 - INFO - bbox_head.shared_conv.1.running_var            loaded from bbox_head.shared_conv.1.running_var                        


***************************model is build_from_cfg***************************
obj_type is str:  RPN
[In build_from_cfg] building  <class 'det3d.models.necks.rpn.RPN'>

***************************model is build_from_cfg***************************
obj_type is str:  CenterHead
[In build_from_cfg] building  <class 'det3d.models.bbox_heads.center_head.CenterHead'>
Use HM Bias:  -2.19
load_checkpoint
Init weight from /home/rajeev-gupta/sensyn_ws/src/GraphRCNN/work_dirs/waymo_centerpoint_voxelnet_graphrcnn_6epoch_freeze/latest.pth
Freeze First Stage Network

***************************model is build_from_cfg***************************
obj_type is str:  GraphRCNNHead
[In build_from_cfg] building  <class 'det3d.models.roi_heads.roi_head.GraphRCNNHead'>


In [15]:
device = torch.device('cpu')
if torch.cuda.is_available():
    device = torch.device('cuda:0')

In [16]:
ckpt = torch.load('/home/rajeev-gupta/sensyn_ws/src/GraphRCNN/work_dirs/waymo_centerpoint_voxelnet_graphrcnn_6epoch_freeze/latest.pth', map_location=device)

In [17]:
ckpt.keys()

dict_keys(['meta', 'state_dict', 'optimizer'])

In [18]:
s_dict = ckpt["state_dict"]
s_dict.keys()

odict_keys(['single_det.backbone.conv_input.0.weight', 'single_det.backbone.conv_input.1.weight', 'single_det.backbone.conv_input.1.bias', 'single_det.backbone.conv_input.1.running_mean', 'single_det.backbone.conv_input.1.running_var', 'single_det.backbone.conv1.0.conv1.weight', 'single_det.backbone.conv1.0.conv1.bias', 'single_det.backbone.conv1.0.bn1.weight', 'single_det.backbone.conv1.0.bn1.bias', 'single_det.backbone.conv1.0.bn1.running_mean', 'single_det.backbone.conv1.0.bn1.running_var', 'single_det.backbone.conv1.0.conv2.weight', 'single_det.backbone.conv1.0.conv2.bias', 'single_det.backbone.conv1.0.bn2.weight', 'single_det.backbone.conv1.0.bn2.bias', 'single_det.backbone.conv1.0.bn2.running_mean', 'single_det.backbone.conv1.0.bn2.running_var', 'single_det.backbone.conv1.1.conv1.weight', 'single_det.backbone.conv1.1.conv1.bias', 'single_det.backbone.conv1.1.bn1.weight', 'single_det.backbone.conv1.1.bn1.bias', 'single_det.backbone.conv1.1.bn1.running_mean', 'single_det.backbone.c

In [19]:
s_dict.keys()

odict_keys(['single_det.backbone.conv_input.0.weight', 'single_det.backbone.conv_input.1.weight', 'single_det.backbone.conv_input.1.bias', 'single_det.backbone.conv_input.1.running_mean', 'single_det.backbone.conv_input.1.running_var', 'single_det.backbone.conv1.0.conv1.weight', 'single_det.backbone.conv1.0.conv1.bias', 'single_det.backbone.conv1.0.bn1.weight', 'single_det.backbone.conv1.0.bn1.bias', 'single_det.backbone.conv1.0.bn1.running_mean', 'single_det.backbone.conv1.0.bn1.running_var', 'single_det.backbone.conv1.0.conv2.weight', 'single_det.backbone.conv1.0.conv2.bias', 'single_det.backbone.conv1.0.bn2.weight', 'single_det.backbone.conv1.0.bn2.bias', 'single_det.backbone.conv1.0.bn2.running_mean', 'single_det.backbone.conv1.0.bn2.running_var', 'single_det.backbone.conv1.1.conv1.weight', 'single_det.backbone.conv1.1.conv1.bias', 'single_det.backbone.conv1.1.bn1.weight', 'single_det.backbone.conv1.1.bn1.bias', 'single_det.backbone.conv1.1.bn1.running_mean', 'single_det.backbone.c

In [12]:
s_dict['single_det.backbone.conv_input.0.weight']

tensor([[[[[ 3.0645e-02,  1.2295e-02, -6.2450e-02, -4.0380e-02,  2.6343e-03],
           [ 2.4632e-02,  2.6060e-02, -3.1059e-03,  6.5041e-02,  2.8911e-02],
           [-3.7599e-02,  7.9337e-02, -3.6656e-02, -5.4235e-02, -7.1812e-02]],

          [[-6.1355e-02,  6.0512e-02, -5.5351e-02, -2.2792e-03, -1.0926e-02],
           [ 5.8681e-02,  3.1912e-02,  6.3133e-02, -2.0601e-02, -1.3058e-04],
           [-4.6652e-03,  1.0976e-02, -1.4378e-02, -8.0774e-02, -7.8107e-02]],

          [[ 4.3298e-02, -6.7924e-02,  2.5508e-02,  7.3475e-02,  6.7547e-02],
           [-4.3813e-02,  2.6662e-02,  8.6400e-03, -4.5210e-02, -8.3933e-02],
           [-1.4712e-02, -7.3552e-02,  1.3297e-02, -7.5289e-02, -6.5377e-02]]],


         [[[ 8.5580e-02, -6.4725e-02,  5.8120e-02, -9.1383e-03, -6.2734e-02],
           [-1.9895e-02, -5.6350e-02,  3.4313e-02,  6.0415e-02, -6.9165e-02],
           [-6.5042e-02, -2.1928e-02,  4.3583e-02, -3.8226e-02, -6.8182e-02]],

          [[-8.1596e-02, -7.9288e-02,  1.4745e-02, -2.

In [13]:

if args.testset:
    print("Use Test Set")
    dataset = build_dataset(cfg.data.test)
else:
    print("Use Val Set")
    dataset = build_dataset(cfg.data.val)


Use Val Set


NameError: name 'build_dataset' is not defined

In [ ]:
type(dataset)

In [ ]:

data_loader = build_dataloader(
    dataset,
    batch_size=cfg.data.samples_per_gpu if not args.speed_test else 1,
    workers_per_gpu=cfg.data.workers_per_gpu,
    dist=distributed,
    shuffle=True,
)


In [ ]:

# checkpoint = load_checkpoint(model, args.checkpoint, map_location="cpu")

# put model on gpus
if distributed:
    model = torch.nn.SyncBatchNorm.convert_sync_batchnorm(model)
    model = DistributedDataParallel(
        model.cuda(cfg.local_rank),
        device_ids=[cfg.local_rank],
        output_device=cfg.local_rank,
        # broadcast_buffers=False,
        find_unused_parameters=False,
    )
else:
    # model = fuse_bn_recursively(model)
    model = model.cuda()

model.eval()
mode = "val"


In [ ]:

logger.info(f"work dir: {args.work_dir}")
if cfg.local_rank == 0:
    prog_bar = torchie.ProgressBar(len(data_loader.dataset) // cfg.gpus)

detections = {}
cpu_device = torch.device("cpu")

start = time.time()

start = int(len(dataset) / 3)
end = int(len(dataset) * 2 /3)

time_start = 0 
time_end = 0 

print('\ndataloader type: ', type(data_loader))


In [ ]:
dataset.__len__()

In [ ]:

# for i, data_batch in enumerate(data_loader):
#     if i == start:
#         torch.cuda.synchronize()
#         time_start = time.time()

#     if i == end:
#         torch.cuda.synchronize()
#         time_end = time.time()

#     with torch.no_grad():
#         outputs = batch_processor(
#             model, data_batch, train_mode=False, local_rank=args.local_rank,
#         )
#     for output in outputs:
#         token = output["metadata"]["token"]
#         for k, v in output.items():
#             if k not in [
#                 "metadata",
#             ]:
#                 output[k] = v.to(cpu_device)
#         detections.update(
#             {token: output,}
#         )
#         if args.local_rank == 0:
#             prog_bar.update()

# synchronize()


In [ ]:

# all_predictions = all_gather(detections)

# print("\n Total time per frame: ", (time_end -  time_start) / (end - start))

# if args.local_rank != 0:
#     return

# predictions = {}
# for p in all_predictions:
#     predictions.update(p)

# if not os.path.exists(args.work_dir):
#     os.makedirs(args.work_dir)

# save_pred(predictions, args.work_dir)

# result_dict, _ = dataset.evaluation(copy.deepcopy(predictions), output_dir=args.work_dir, testset=args.testset)

# if result_dict is not None:
#     for k, v in result_dict["results"].items():
#         print(f"Evaluation {k}: {v}")

# if args.txt_result:
#     assert False, "No longer support kitti"


In [ ]:
# # learning to implement decorators

# import time

# class timing_decorator(object):
#     def __init__(self) -> None:
#         pass
    
#     def wrapper(*args, **kwargs):
#         start_time = time.time()
#         print(args)
#         obj = args[1]()
#         time.sleep(1)
#         end_time = time.time()
#         print(f"execution took {end_time - start_time:.4f} seconds")
#         return args
    
# td = timing_decorator()


In [ ]:
# @td.wrapper
# class new_class():
#     def __init__(self):
#         time.sleep(2)
#         print("hey there")
